In [1]:
import pandas as pd
import numpy as np

Notes:
* They tested 321, only 2 Abs failed to bind to GPdMuc, indicating that these bind to the mucin-like domain
* This means that they all should bind the overall GP
* Since I can't find the joined sepcific seqs they used for BLI, and the PlabDab has different naming for the Abs, it will make the most sense to just use the same strain GP for all seqs

In [2]:
# https://www.ncbi.nlm.nih.gov/nuccore/KJ660347
gp = 'MGVTGILQLPRDRFKRTSFFLWVIILFQRTFSIPLGVIHNSTLQVSDVDKLVCRDKLSSTNQLRSVGLNLEGNGVATDVPSATKRWGFRSGVPPKVVNYEAGEWAENCYNLEIKKPDGSECLPAAPDGIRGFPRCRYVHKVSGTGPCAGDFAFHKEGAFFLYDRLASTVIYRGTTFAEGVVAFLILPQAKKDFFSSHPLREPVNATEDPSSGYYSTTIRYQATGFGTNETEYLFEVDNLTYVQLESRFTPQFLLQLNETIYASGKRSNTTGKLIWKVNPEIDTTIGEWAFWETKKNLTRKIRSEELSFTAVSNGPKNISGQSPARTSSDPETNTTNEDHKIMASENSSAMVQVHSQGRKAAVSHLTTLATISTSPQPPTTKTGPDNSTHNTPVYKLDISEATQVGQHHRRADNDSTASDTPPATTAAGPLKAENTNTSKSADSLDLATTTSPQNYSETAGNNNTHHQDTGEESASSGKLGLITNTIAGVAGLITGGRRTRREVIVNAQPKCNPNLHYWTTQDEGAAIGLAWIPYFGPAAEGIYTEGLMHNQDGLICGLRQLANETTQALQLFLRATTELRTFSILNRKAIDFLLQRWGGTCHILGPDCCIEPHDWTKNITDKIDQIIHDFVDKTLPDQGDNDNWWTGWRQWIPAGIGVTGVIIAVIALFCICKFVF'
print(len(gp))

gpTM = gp[:650]

# https://www.ncbi.nlm.nih.gov/nuccore/KJ660347
sGP = 'MGVTGILQLPRDRFKRTSFFLWVIILFQRTFSIPLGVIHNSTLQVSDVDKLVCRDKLSSTNQLRSVGLNLEGNGVATDVPSATKRWGFRSGVPPKVVNYEAGEWAENCYNLEIKKPDGSECLPAAPDGIRGFPRCRYVHKVSGTGPCAGDFAFHKEGAFFLYDRLASTVIYRGTTFAEGVVAFLILPQAKKDFFSSHPLREPVNATEDPSSGYYSTTIRYQATGFGTNETEYLFEVDNLTYVQLESRFTPQFLLQLNETIYASGKRSNTTGKLIWKVNPEIDTTIGEWAFWETKKTSLEKFAVKSCLSQLYQTDPKTSVVRVRRELLPTQRPTQQMKTTKSWLQKIPLQWFKCTVKEGKLQCRI'

676


In [3]:
binding_table = pd.read_csv('./bornholdt_binding_table_24-01-02.csv', header=[1,2], index_col=0)

# First sheet of their provided table has ID link between GenBank and binding data
id_table = pd.read_excel('./bornholdt_binding_table_24-01-02.xlsx', header=2, index_col=0)
id_table= id_table[[i for i in id_table if 'Unnamed' not in i]]

In [4]:
# Pulled nucelotide sequences from GenBank and ran through IMGT
imgt_seqs = pd.read_csv('./bornholdt_hc_lc_seqs_IMGT/5_AA-sequences.txt', sep='\t', index_col=0)

# Getting sequence ID that aligns with binding table
imgt_seqs['Sequence ID'] = imgt_seqs['Sequence ID'].apply(lambda x: x.split('|')[1].split('.')[0])

In [5]:
# Mapping in amino acid sequences
id_table['VH_AA'] = id_table['VH'].map(dict(zip(imgt_seqs['Sequence ID'], imgt_seqs['V-D-J-REGION'])))
id_table['VL_AA'] = id_table['VL'].map(dict(zip(imgt_seqs['Sequence ID'], imgt_seqs['V-J-REGION'])))
id_table = id_table.dropna()


seq_df = id_table.copy()

In [6]:
binding_table.columns = ['VH Germline', 'VH CDR3',
            'Zaire GP∆TM: IgG K_{D} (M)*',
           'kon (1/Ms)','koff (1/s)^{†}',
                   'Zaire GP∆muc: IgG K_{D} (M)*',
            'kon (1/Ms)', 'koff (1/s)^{†}', 'Zaire sGP: IgG K_{D} (M)*',
            'kon (1/Ms)', 'koff (1/s)^{†}']

binding_table = binding_table.drop([i for i in binding_table if 'ko' in i], axis=1)

In [7]:
binding_cols = [ 'Zaire GP∆TM: IgG K_{D} (M)*', 'Zaire sGP: IgG K_{D} (M)*'] #'Zaire GP∆muc: IgG K_{D} (M)*',  Couldn't find a good reference for this sequence

binders = {}
for col in binding_cols:
    binding_table[col] = pd.to_numeric(binding_table[col], errors='coerce')

binding_table['strongest_binder'] = binding_table.apply(lambda x: (x[binding_cols].astype(float).idxmin()), axis=1)


In [8]:
binding_table['Antigen'] = binding_table['strongest_binder'].apply(lambda x: str(x).split(':')[0])
binding_table['Antigen'].value_counts()

Zaire sGP      191
Zaire GP∆TM    103
nan             27
Name: Antigen, dtype: int64

In [9]:
antigen_map = {'Zaire sGP':sGP, 'Zaire GP∆TM':gpTM}

binding_table['antigen_seq'] = binding_table['Antigen'].map(antigen_map)

In [10]:
binding_table = binding_table[~binding_table['antigen_seq'].isna()]

# All are under 1e-7 M binding

In [22]:
binding_table[binding_cols].apply(max, axis=1).max()

9.73e-08

In [14]:
binding_table['VH_AA'] = seq_df['VH_AA']
binding_table['VL_AA'] = seq_df['VL_AA']

In [16]:
binding_table.to_csv('bornholdt_binders_v2_24-03-07.csv')

In [161]:
# seq_df[['Antigen', 'antigen_seq']] = binding_table[['Antigen', 'antigen_seq']]

In [166]:
# seq_df.to_csv('bornholdt_binders_v2_24-03-07.csv')